In [1]:
# 生成线性回归的数据

In [2]:
import numpy as np
import random
import pandas as pd
import math

In [23]:
import os
project_path = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('.')), '../'))
import sys
sys.path.append(project_path)
from util.visualization import draw_lines
from util.visualization import draw_scatters
from util.evaluate_process import mae, mse, rmse

In [18]:
n_d = 10
n_data = 1000
constant_val = 13
### 离群点
outlier_rate = 0.03
###
darr = [round(random.random() * 20-8, 1) for i in range(n_d)]
data = {f'd{i}': 
        [round(random.random() * 100+i*10-50, 2) for j in range(n_data)] 
        for i in range(n_d)}


df = pd.DataFrame(data)
#df['b'] = constant_val
df['y'] = df.apply(lambda row: np.dot(list(row), darr)+constant_val+ (random.random()*20-10)*(1 if random.random()>outlier_rate else 13), axis=1)

In [19]:
df.head()

,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,y
0,36.81,25.09,47.98,33.46,-5.32,27.67,98.92,45.85,68.12,139.28,705.905389
1,-48.88,12.36,-24.02,0.42,26.45,25.71,14.52,42.39,45.33,87.36,698.050785
2,-28.59,3.34,35.34,39.56,51.35,13.36,20.59,80.20,65.51,102.63,-57.348093
3,16.37,-10.79,4.47,56.09,34.14,72.48,41.76,82.41,45.08,64.42,136.479240
4,-18.86,-28.85,27.55,58.26,19.57,26.85,50.16,110.60,71.63,98.36,-199.452840


In [20]:
darr

[-5.4, 9.3, -0.6, -3.1, -7.4, 9.9, 0.5, -1.7, -6.8, 6.9]

In [21]:
## 归一化
cf = df.copy()
mins = {k: min(df[k]) for k in df.columns}
maxs = {k: max(df[k]) for k in df.columns}
widths = {k: maxs[k] -mins[k] for k in df.columns}
###
processed_darr = [val*widths[f'd{i}']/widths['y'] for i,val in enumerate(darr)]
processed_b = (np.dot(list(mins.values()), darr + [-1]) + constant_val ) / widths['y']
###
for k in cf.columns:
    cf[k] = cf[k].map(lambda x: (x-mins[k])/widths[k])
cf.head()

,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,y
0,0.868495,0.650616,0.781523,0.533774,0.046328,0.276758,0.889990,0.258394,0.381731,0.995786,0.675993
1,0.010909,0.523175,0.059284,0.202646,0.364219,0.257097,0.045145,0.223715,0.153214,0.474867,0.673626
2,0.213971,0.432876,0.654730,0.594909,0.613368,0.133213,0.105906,0.602686,0.355560,0.628073,0.446011
3,0.663931,0.291421,0.345070,0.760573,0.441165,0.726251,0.317818,0.624837,0.150707,0.244708,0.504415
4,0.311349,0.110622,0.576587,0.782321,0.295377,0.268532,0.401902,0.907387,0.416926,0.585231,0.403193


In [39]:
### 证明归一化没问题
X = list(cf.apply(lambda row: list(row)[:-1]+[1], axis=1))  # m x d+1
pred = np.dot(X, processed_darr + [processed_b])
print(mse(cf.y, pred), rmse(cf.y, pred), mae(cf.y, pred))
edf = pd.DataFrame({'target': cf.y, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(edf))], [edf.target, edf.pred], ['true', 'pred'])
list(cf.y)[:5], pred[:5]

5.740954283670896e-05 0.007576908527672018 0.0034259555130648966


([0.23635360897379692,
  0.5783060564849531,
  0.5667247255589608,
  0.6556958784589503,
  0.26546416622564606],
 array([0.23772192, 0.57768308, 0.56628687, 0.66013544, 0.26377873]))

In [25]:
df.to_csv('linear_model_data.10d.v1.csv', index=None)

In [26]:
params = pd.DataFrame([darr + [constant_val], processed_darr+[processed_b] ], columns=[f'd{i}' for i in range(n_d)] + ['b'])
params

,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,b
0,-5.400000,9.300000,-0.600000,-3.100000,-7.400000,9.90000,0.500000,-1.700000,-6.800000,6.900000,13.000000
1,-0.162581,0.279917,-0.018023,-0.093203,-0.222841,0.29738,0.015051,-0.051106,-0.204343,0.207223,0.495918


In [27]:
params.to_csv('linear_model_data.params.10d.v1.csv', index=None)

### 这里用GD 去拟合

In [28]:
def mse(target, pred):
    return np.mean([(p-t)**2 for p,t in zip(pred, target)])

def msegd(target, pred, XT):
    error = [(p-t) for p,t in zip(pred, target)]
    return np.dot(XT, error) / len(pred) * 2

In [33]:

karr = np.array([1 for i in range(n_d + 1)] ) # d+1 x 1
gdrate = [1 for i in range(n_d + 1)]  # d+1 x 1
karrs = [karr]
karr2 = [[] for i in range(n_d + 1)]
msearr = []
###
ydata = cf.y 

xf = cf[list(cf.columns)[:-1]]
xf['cn'] = 1
xf.head()
####
run_times = [1]
run_epoch = 0
####


In [34]:
eta = 0.3

X = list(xf.apply(lambda row: list(row), axis=1))  # m x d+1
XT = np.transpose(X)
for i in range(700):
    run_epoch += 1
    hy = np.dot(X, karr)
    ###
    gd = msegd(ydata, hy, XT)
    ### 随梯度的变化而变化, 随时间变化而变化
    if karr2[0]:
        tau = math.log10(run_epoch+11)
        gdrate = [ np.mean(_gdarr) ** 0.5 * tau for _gdarr in karr2  ] 
    karr = karr - eta * gd / gdrate
    ###
    if i % 1 == 0:
        karrs.append(karr)
        run_times.append(run_epoch)
        msearr.append(mse(ydata, hy))
    
    for si,gdv in enumerate(gd):
        karr2[si].append(gdv ** 2++ 1e-10)
    
    

hy = np.dot(X, karr)
msearr.append(mse(ydata, hy))
##
trans_karrs = np.transpose(karrs)
## 展示最后两位参数的变化
draw_scatters([list(trans_karrs[-2]), [processed_darr[-1]]], [list(trans_karrs[-1]), [processed_b]], ['b', 'true'])
draw_lines(run_times, [msearr], ['mse'])
###
draw_lines(run_times, np.transpose(karrs), [i for i in range(n_d + 1)])